# Tutorial 02: Custom Hamiltonian

Learn how to define custom MFG problems by implementing your own Hamiltonian and problem components.

## Learning Objectives

- Understand the `MFGProblem` abstract class
- Implement custom Hamiltonians with congestion effects
- Define terminal costs and initial densities
- Work with the fundamental MFG equations

## Mathematical Formulation

We'll solve a crowd evacuation problem where agents want to reach a safe zone while avoiding congestion:

**HJB equation** (backward in time):
$$-\frac{\partial u}{\partial t} + H(x, p, m) = 0$$

**Fokker-Planck equation** (forward in time):
$$\frac{\partial m}{\partial t} - \sigma^2 \frac{\partial^2 m}{\partial x^2} + \frac{\partial}{\partial x}\left(m \frac{\partial H}{\partial p}\right) = 0$$

**Hamiltonian** with congestion:
$$H(x, p, m) = \frac{1}{2}|p|^2 + \kappa \cdot m \cdot |p|^2$$

**Time estimate**: 15 minutes

## Step 1: Import Dependencies

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from mfg_pde import MFGProblem, solve_mfg

## Step 2: Define Custom Problem Class

We subclass `MFGProblem` and implement:
- `hamiltonian()`: The running cost function
- `hamiltonian_dm()`: Derivative of H with respect to density
- `terminal_cost()`: Penalty at final time
- `initial_density()`: Starting distribution of agents

In [ ]:
class CrowdEvacuationMFG(MFGProblem):
    """Crowd evacuation with congestion-dependent control cost."""

    def __init__(self, congestion_strength=1.0):
        # Define 1D domain [0, 1] with 60 points
        # Time horizon [0, 1] with 60 timesteps
        # Diffusion coefficient σ = 0.05
        super().__init__(xmin=0.0, xmax=1.0, Nx=60, T=1.0, Nt=60, sigma=0.05)
        self.congestion_strength = congestion_strength

    def hamiltonian(self, x, p, m, t):
        """H = (1/2)|p|² + κ·m·|p|²

        Standard control cost plus congestion-dependent increase.
        High density m makes control more expensive.
        """
        standard_cost = 0.5 * p**2
        congestion_cost = self.congestion_strength * m * p**2
        return standard_cost + congestion_cost

    def hamiltonian_dm(self, x, p, m, t):
        """∂H/∂m = κ·|p|²

        Derivative needed for HJB coupling term.
        """
        return self.congestion_strength * p**2

    def terminal_cost(self, x):
        """g(x) = (x - 1)²

        Agents want to reach x = 1 (safe zone).
        """
        return (x - 1.0) ** 2

    def initial_density(self, x):
        """m₀(x) = uniform on [0.1, 0.9]

        Agents start uniformly distributed.
        """
        density = np.where((x >= 0.1) & (x <= 0.9), 1.0, 0.0)
        # Normalize to ensure ∫m dx = 1
        return density / np.trapz(density, x)


print("Custom problem class defined!")

## Step 3: Solve Without Congestion

First, solve with no congestion (κ = 0) to establish baseline behavior.

In [ ]:
problem_no_congestion = CrowdEvacuationMFG(congestion_strength=0.0)
print("Solving without congestion (κ = 0)...\n")
result_no_congestion = solve_mfg(problem_no_congestion, verbose=True)

print(f"\nConverged: {result_no_congestion.converged}")
print(f"Iterations: {result_no_congestion.iterations}")

## Step 4: Solve With Congestion

Now solve with congestion effects (κ = 1.0).

In [ ]:
problem_with_congestion = CrowdEvacuationMFG(congestion_strength=1.0)
print("Solving with congestion (κ = 1.0)...\n")
result_with_congestion = solve_mfg(problem_with_congestion, verbose=True)

print(f"\nConverged: {result_with_congestion.converged}")
print(f"Iterations: {result_with_congestion.iterations}")

## Step 5: Compare Results

Visualize how congestion affects the solution.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

x = problem_no_congestion.xSpace
t = problem_no_congestion.tSpace

# Density at t=0 (initial)
axes[0, 0].plot(x, result_no_congestion.M[0, :], "b-", label="No congestion", linewidth=2)
axes[0, 0].plot(x, result_with_congestion.M[0, :], "r--", label="With congestion", linewidth=2)
axes[0, 0].set_xlabel("x")
axes[0, 0].set_ylabel("Density m(0, x)")
axes[0, 0].set_title("Initial Density")
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Density at t=T (final)
axes[0, 1].plot(x, result_no_congestion.M[-1, :], "b-", label="No congestion", linewidth=2)
axes[0, 1].plot(x, result_with_congestion.M[-1, :], "r--", label="With congestion", linewidth=2)
axes[0, 1].set_xlabel("x")
axes[0, 1].set_ylabel("Density m(T, x)")
axes[0, 1].set_title("Final Density")
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Value function at t=0
axes[1, 0].plot(x, result_no_congestion.U[0, :], "b-", label="No congestion", linewidth=2)
axes[1, 0].plot(x, result_with_congestion.U[0, :], "r--", label="With congestion", linewidth=2)
axes[1, 0].set_xlabel("x")
axes[1, 0].set_ylabel("Value u(0, x)")
axes[1, 0].set_title("Initial Value Function")
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Density evolution (with congestion)
X, T = np.meshgrid(x, t)
contour = axes[1, 1].contourf(X, T, result_with_congestion.M, levels=20, cmap="viridis")
axes[1, 1].set_xlabel("x")
axes[1, 1].set_ylabel("t")
axes[1, 1].set_title("Density Evolution (With Congestion)")
plt.colorbar(contour, ax=axes[1, 1])

plt.tight_layout()
plt.show()

## Step 6: Analyze Congestion Effects

In [ ]:
print("=" * 70)
print("CONGESTION EFFECTS ANALYSIS")
print("=" * 70)
print()

# Compare final densities at target x = 1.0
m_final_no_cong = result_no_congestion.M[-1, -1]
m_final_with_cong = result_with_congestion.M[-1, -1]

print("Final density at target (x=1.0):")
print(f"  No congestion:   m(T, 1.0) = {m_final_no_cong:.4f}")
print(f"  With congestion: m(T, 1.0) = {m_final_with_cong:.4f}")
print()

# Compare value functions (cost to reach target)
u_initial_no_cong = np.mean(result_no_congestion.U[0, :])
u_initial_with_cong = np.mean(result_with_congestion.U[0, :])

print("Average initial cost:")
print(f"  No congestion:   u(0, x) = {u_initial_no_cong:.4f}")
print(f"  With congestion: u(0, x) = {u_initial_with_cong:.4f}")
print(f"  Increase: {(u_initial_with_cong - u_initial_no_cong) / u_initial_no_cong * 100:.1f}%")
print()

print("Interpretation:")
print("  - Congestion increases optimal cost (higher u)")
print("  - Agents spread out more to avoid crowding")
print("  - Final density at target is reduced")

## Summary

### What You Learned

1. How to subclass `MFGProblem` to define custom problems
2. How to implement Hamiltonians with congestion effects
3. How to define terminal costs and initial densities
4. How congestion affects optimal behavior and equilibrium

### Key Concepts

- **Hamiltonian**: Encodes running cost and control strategy
- **Congestion coupling**: Higher density makes control more expensive
- **Terminal cost**: Defines objective (reach target location)
- **Initial density**: Specifies agent distribution at t=0

### Mathematical Insight

The congestion term $\kappa \cdot m \cdot |p|^2$ in the Hamiltonian creates a feedback loop:
- High density $m$ increases control cost
- Agents slow down in crowded regions
- This spreads out the density, reducing congestion

### Next Steps

Proceed to **Tutorial 03: 2D Geometry** to learn about multi-dimensional problems.